In [1]:
from descarteslabs import workflows as wf
import ipyleaflet
import ipywidgets
import matplotlib.pyplot as plt
import descarteslabs as dl
import descarteslabs.workflows as wf
from shapely import geometry as sgeom
import logging
import will_utils as utils
import pandas as pd
logging.basicConfig(level=logging.ERROR)

/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:57: FutureWarning: 'pyarrow.SerializationContext' is deprecated and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  serialization_context = pa.SerializationContext()
/opt/conda/lib/python3.7/site-packages/descarteslabs/common/workflows/arrow_serialization/context.py:58: FutureWarning: 'pyarrow.register_default_serialization_handlers' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  pa.register_default_serialization_handlers(serialization_context)


In [2]:
import geopandas as gpd

In [3]:
#import yolo country
tomato_aoi = gpd.read_file('yolo_county.geojson')
aoi = {'year': [2018, 2019,2020], 'acres tomato': [246, 290,382]}
acreage_out = pd.DataFrame(aoi)

In [4]:
s2_A = wf.ImageCollection.from_id('sentinel-2:L1C',
                                   start_datetime='2019-08-01',
                                   end_datetime='2019-08-31', processing_level="surface"
                                   ).pick_bands('red green blue cloud-mask')

s2_B = wf.ImageCollection.from_id('sentinel-2:L1C',
                                   start_datetime='2018-08-01',
                                   end_datetime='2018-08-31',  processing_level="surface"
                                   ).pick_bands('red green blue cloud-mask')

s2_C = wf.ImageCollection.from_id('sentinel-2:L1C',
                                   start_datetime='2017-08-01',
                                   end_datetime='2017-08-31',  processing_level="surface"
                                   ).pick_bands('red green blue cloud-mask')

s2_D = wf.ImageCollection.from_id('sentinel-2:L1C',
                                   start_datetime='2020-07-01',
                                   end_datetime='2020-07-31',  processing_level="surface"
                                   ).pick_bands('red green blue').mosaic()


cmaskA = s2_A.pick_bands("cloud-mask") > 0
cmaskB = s2_B.pick_bands("cloud-mask") > 0 
cmaskC = s2_C.pick_bands("cloud-mask") > 0 
cmaskD = s2_D.pick_bands("cloud-mask") > 0

#Okay at this point I should obviously have used an iterable
s2_A = s2_A.mask(cmaskA).pick_bands('red green blue')
s2_B = s2_B.mask(cmaskB).pick_bands('red green blue')
s2_C = s2_C.mask(cmaskC).pick_bands('red green blue')
s2_D = s2_D.mask(cmaskD).pick_bands('red green blue')

In [5]:
product_id = "sentinel-1:sar:sigma0v:v1"

# Define simag0 image Collection
sigA = (wf.ImageCollection.from_id(product_id, start_datetime='2019-07-01', end_datetime='2019-07-31',).pick_bands("vv vh"))
sigB = (wf.ImageCollection.from_id(product_id, start_datetime='2018-07-01', end_datetime='2018-07-31',).pick_bands("vv vh"))
sigC = (wf.ImageCollection.from_id(product_id, start_datetime='2017-07-01', end_datetime='2017-07-31',).pick_bands("vv vh"))
sigD = (wf.ImageCollection.from_id(product_id, start_datetime='2021-01-01', end_datetime='2021-01-31',).pick_bands("vv vh"))


In [6]:
# Various CDLs
cdl_2020 = wf.ImageCollection.from_id("usda:cdl:v1", start_datetime="2019-12-31", end_datetime="2020-01-01").mosaic()
cdl_2019 = wf.ImageCollection.from_id("usda:cdl:v1", start_datetime="2018-12-31", end_datetime="2019-01-01").mosaic()
cdl_2018 = wf.ImageCollection.from_id("usda:cdl:v1", start_datetime="2017-12-31", end_datetime="2018-01-01").mosaic()

cdl_2020 = cdl_2020.mask(cdl_2020 != 54)
cdl_2019 = cdl_2019.mask(cdl_2019 != 54)
cdl_2018 = cdl_2018.mask(cdl_2018 != 54)

In [7]:
# Create the map
m1 = None
m1 = wf.interactive.MapApp()
m1.center = (38.7751, -121.7282) #
m1.zoom = 13
m1.control_other_layers = True
#m1.map.layout.height = "600px"

In [8]:
m1.clear_layers()
#Add my vector layer
yolo = ipyleaflet.GeoData(
    geo_dataframe = tomato_aoi,
    style={"color": "blue", "fillOpacity":0.},
    name="Yolo County",
)
m1.add_layer(yolo)

In [9]:
## Sentinel 2
#s2_D.visualize(name="Sentinel 2, July 2021", map=m1, scales=[[0,0.5], [0,0.5], [0,0.5]], checkerboard=False)


s2_A.visualize(name="Sentinel 2, July 2020", map=m1, scales=[[0,0.5], [0,0.5], [0,0.5]], checkerboard=False)
s2_B.visualize(name="Sentinel 2, July 2019", map=m1, scales=[[0,0.5], [0,0.5], [0,0.5]], checkerboard=False)
#s2_C.visualize(name="Sentinel 2, July 2018", map=m1, scales=[[0,0.5], [0,0.5], [0,0.5]], checkerboard=False)



In [10]:
# Sentinel 1
sigA.pick_bands("vv vh vv").visualize("Sentinel 1, July 2020", scales=[[-30,30],[-30,30],[-30,30]], map=m1)
sigB.pick_bands("vv vh vv").visualize("Sentinel 1, July 2019", scales=[[-30,30],[-30,30],[-30,30]], map=m1)
#sigC.pick_bands("vv vh vv").visualize("Sentinel 1, July 2018", scales=[[-30,30],[-30,30],[-30,30]], map=m1)
#sigD.pick_bands("vv vh vv").visualize("Sentinel 1, July 2021", scales=[[-30,30],[-30,30],[-30,30]], map=m1)

In [11]:
## Add my raster layers
## CDL
cdl_2020.visualize(name="2020 CDL Tomatoes", colormap='Greens', checkerboard=False, map=m1)
cdl_2019.visualize(name="2019 CDL Tomatoes", colormap='Purples', checkerboard=False, map=m1)
cdl_2018.visualize(name="2018 CDL Tomatoes", colormap='Reds', checkerboard=False, map=m1)



In [12]:
from acreage import acreage

acreage(m1)

In [13]:

m1


`ipyleaflet` and/or `ipywidgets` Jupyter extensions are not installed! (or you're not in a Jupyter notebook.)
To install for JupyterLab, run this in a cell:
    !jupyter labextension install jupyter-leaflet @jupyter-widgets/jupyterlab-manager
To install for plain Jupyter Notebook, run this in a cell:
    !jupyter nbextension enable --py --sys-prefix ipyleaflet
Then, restart the kernel and refresh the webpage.


In [14]:
## Print the AOI 
display(acreage_out)

,year,acres tomato
0,2018,246
1,2019,290
2,2020,382
